In [1]:
import matplotlib
from matplotlib import font_manager, rc
font_path = 'C:/Windows/Fonts/malgun.ttf'
font_name = font_manager.FontProperties(fname=font_path).get_name()
matplotlib.rc('font',family=font_name)

- matplotlib의 rc는 "runtime configuration"의 약자로, 그래픽의 스타일과 동작을 설정하는 데 사용되는 구성 매개변수


In [2]:
import os
import pandas as pd

path = 'D:\elice_python\GAS_5\pytest'

os.chdir(path)
os.getcwd()

'D:\\elice_python\\GAS_5\\pytest'

In [3]:
def read_data(filename:str, encoding='cp949', start=1):
    with open(filename, 'r', encoding=encoding) as f :
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[start:]  # 파일 header 제외를 위해 설정.
    return data  # 각 요소는 파일의 한 줄에 해당하며, 각 줄은 탭으로 구분된 값들로 이루어진 리스트

In [4]:
def write_data(data,filename:str,encoding='cp949'):
    with open(filename,'w',encoding=encoding) as f :
        f.write(data)

In [5]:
data = read_data('ratings_morphed.txt', encoding='cp949',start=0)
print(len(data))
print(len(data[0]))
data_text = [line[1] for line in data]
data_senti = [line[2] for line in data]

495
3


# n-gram search
- 모든 단어의 연결을 고려하는 것은 어렵지만 2개 혹은 3개 단어가 연결된 정도를 보는 것은 비교적 쉽다.
	- 일종의 문맥을 참고하는 방식
	- 관용 표현을 파악할 수도 있다 (물 흐르듯 한다, 밑도 끝도 없다)
- 유니그램(uni-gram, 1개 단어), 바이그램(bi-gram, 2개 단어), 트라이그램(tri-gram, 3개 단어)이라고 하며, 통칭 n-그램이라고 한다

- n-그램 검색은 scikit-learn의 CountVectorizer의 ngram_range 파라미터로 가능하다
	- ngram_range=(1, 1) : 유니그램
	- ngram_range=(2, 2) : 바이그램
	- ngram_range=(3, 3) : 트라이그램
	- ngram_range=(1, 3) : 유니그램, 바이그램, 트라이그램


In [6]:
test_words = ['this is the great', 'yes this is the great!']

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(2,2)).fit(test_words)
vect.get_feature_names_out()

array(['is the', 'the great', 'this is', 'yes this'], dtype=object)

In [9]:
vect = CountVectorizer(min_df=5, ngram_range=(2,2)).fit(data_text)
print(len(vect.vocabulary_))
print(vect.get_feature_names_out())

12
['ㅋㅋ ㅋㅋ' '다시 보다' '만들다 영화' '보다 내내' '보다 영화' '시간 아깝다' '영화 보다' '이런 영화' '재미 없다'
 '정말 재미있다' '좋다 영화' '하다 영화']


- min_df는 CountVectorizer에서 사용되는 매개변수로, "minimum document frequency"의 약자

-  전체 데이터를 형태소 분석한 결과로는 23,467 쌍이 발견되었다
- 이러한 바이너리 쌍의 집합을 bag of 2-gram 이라고 한다

# 문장 분리기

## 문장 분리기 kss

- 한국어를 위한 문장 분리기 라이브러리로 kss를 사용할 수 있다
	-  그러나 이 패키지는 numpy 등 다른 패키지 사용에 문제를 줄 수 있으니 Colab에서만 사용할 것을 추천
- https://docs.likejazz.com/kss/

# 띄어쓰기 교정기
## PyKoSpacing
- 띄어쓰기만을 바르게 해결하는 라이브러리로 PyKoSpacing이 있다
	-  그러나 이 패키지는 numpy 등 다른 패키지 사용에 문제를 줄 수 있다. Colab에서 설치 추천
- 설치: !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
- Colab에서는 !pip install git+https://github.com/haven-jeon/PyKoSpacing.git --no-deps argument

# 음절 및 자소 분리
- 음절 단위 분리는 문자열을 리스트로 분리하는 것으로 바로 가능하고,
-  자소 단위 분리는 jamo 패키지를 사용하는 것으로 가능하다
```
!pip install jamo
from jamo import h2j, j2hcj
review = '안녕하세요, Hi!'
decomposed = list(review)
print(decomposed)
```
- h2j는 완성형 → 조합형 변환, j2hcj는 조합형 문자열을 자소 단위로 분리한다
```
decomposed2 = list( j2hcj(h2j(review)))
print(decomposed2)
```
